In [30]:
%pip install plotly
%pip install nbformat>=4.2.0
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


1.4. a crazy idea followed by a stupid one

In [31]:
# chatgpt helped writing out rules

XT = np.array([
    [+1, +1, +1],
    [+1, +1, -1],
    [+1, -1, +1],
    [+1, -1, -1],
    [-1, +1, +1],
    [-1, +1, -1],
    [-1, -1, +1],
    [-1, -1, -1]
])

y_rule_110 = np.array([+1, -1, -1, -1, +1, -1, -1, +1])

y_rule_126 = np.array([+1, -1, -1, -1, -1, -1, -1, +1])

# Solve the least squares problem for rule 110
w_star_110, residuals_110, rank_110, s_110 = np.linalg.lstsq(XT, y_rule_110, rcond=None)

y_hat_110 = XT @ w_star_110

# Solve the least squares problem for rule 126
w_star_126, residuals_126, rank_126, s_126 = np.linalg.lstsq(XT, y_rule_126, rcond=None)

y_hat_126 = XT @ w_star_126

# Create dataframes for better comparison of y and y_hat
results_110 = pd.DataFrame({
    'Original y (Rule 110)': y_rule_110,
    'Predicted y_hat (Rule 110)': y_hat_110
})

results_126 = pd.DataFrame({
    'Original y (Rule 126)': y_rule_126,
    'Predicted y_hat (Rule 126)': y_hat_126
})

# Print the results in table format for better alignment and comparison
print("Results for Rule 110:")
print(results_110)
print()

print("Results for Rule 126:")
print(results_126)


# Function to visualize points in 3D space for better understanding of non-linearity using Plotly
def plot_3d_representation(XT, y, title):
    # Define colors based on y values
    colors = ['blue' if val == 1 else 'red' for val in y]

    # Create a scatter plot in 3D
    trace = go.Scatter3d(
        x=XT[:, 0],
        y=XT[:, 1],
        z=XT[:, 2],
        mode='markers',
        marker=dict(
            size=10,
            color=colors,
            opacity=0.8,
            symbol='circle'
        ),
        text=[f'y={val}' for val in y]
    )

    layout = go.Layout(
        title=title,
        scene=dict(
            xaxis_title='X1',
            yaxis_title='X2',
            zaxis_title='X3'
        )
    )

    fig = go.Figure(data=[trace], layout=layout)
    pio.show(fig)

# Plotting for Rule 110 and Rule 126
plot_3d_representation(XT, y_rule_110, "Rule 110: 3D Representation")
plot_3d_representation(XT, y_rule_126, "Rule 126: 3D Representation")




Results for Rule 110:
   Original y (Rule 110)  Predicted y_hat (Rule 110)
0                      1                        0.25
1                     -1                       -0.25
2                     -1                       -0.25
3                     -1                       -0.75
4                      1                        0.75
5                     -1                        0.25
6                     -1                        0.25
7                      1                       -0.25

Results for Rule 126:
   Original y (Rule 126)  Predicted y_hat (Rule 126)
0                      1                1.570092e-16
1                     -1                0.000000e+00
2                     -1                1.570092e-16
3                     -1                0.000000e+00
4                     -1                0.000000e+00
5                     -1               -1.570092e-16
6                     -1                0.000000e+00
7                      1               -1.570092e-16


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

They cannot be linear in the 3 arguments, since the values of y hat are significantly off.
